In [6]:
# Set dependencies and import libraries
from splinter import Browser
from bs4 import BeautifulSoup, Comment
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver

import time
import pandas as pd
import datetime
import pymongo
from time import sleep
from random import randint
import re

In [20]:
# Set some variables
dlr_name = ""
date = datetime.date.today()
scraped_url = "https://www.carsguide.com.au/buy-a-car/all-new-and-used/all-states/all-locations/all-bodytypes/all-makes/?searchTypes=USED%20Dealer%20ad,USED%20Private%20ad&distanceFromMe=Perth,WA"

In [21]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(scraped_url)



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\chrispenn\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


In [5]:
# Define clean-up functions
# def clean_name():
#     return dealer_name.replace('\n','')
    # dealer_name = dealer_name.replace('Message','')
    # dealer_name = dealer_name.strip()



2021-10-08


In [22]:
#  Define collection
cars = [] # To store list of dictionaries
pages = 100 # Set the number of pages to scroll through
date = datetime.date.today()
date = date.strftime('%Y/%m/%d')

# Connect to MongoDB
client = pymongo.MongoClient('mongodb://localhost:27017')
db = client['carsguide_used_cars_australia']
col_cars = db['scraped_cars']
col_sub_stats = db['suburb_stats']

# Set for loop to iterate through all pages
for x in range(pages):

    # Set HTML object
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
    # Retrieve all elements that contain listing data
    results = soup.find_all('div', class_='carListing--text')
    # Retrieve all elements that contain dealer name
    dealer_data = soup.find_all('div', class_='cgForm')

    # Iterate through each listing
    for result, dealer in zip(results, dealer_data):
        
        try:
            year_make_model = result.find('h5', class_='carListing--textHeading').text
        except:
            year_make_model = "NULL"
        try:
            km = result.find('li', class_='icon-gauge').text
        except:
            km = "NULL"
        try:
            trans = result.find('li', class_='icon-gears').text
        except:
            trans = "NULL"
        try:
            price = result.find('h6', class_='carListing--textPrice').text
        except:
            price = "NULL"
        try:
            comment = result.find('div', class_='carListing--comments').text
        except:
            comment = "NULL"
        try:
            listing_type = result.find('li', class_='sellerCircle').text
        except:
            listing_type = "NULL"
        try:
            suburb = result.find('li', class_='icon-pin').text
        except:
            suburb = "NULL" 
        try:
            dealer_name = dealer.find('h1').text
        except:
            dealer_name = "NULL"

        # Clean up data
        dealer_name = dealer_name.replace('\n','')
        dealer_name = dealer_name.replace('Message','')
        dealer_name = dealer_name.strip()
        km = km.replace(',','')
        km = km.replace('km','')
        price = price.replace(',','')
        price = price.replace('$','')
        comment = comment.replace('\n','')
        source = scraped_url
        year = year_make_model[:4]
        make = year_make_model.split(' ')[1]
        model = year_make_model.split(' ')[2:]
        state = suburb.split(', ')[1]
        suburb = suburb.split(', ')[0]

        car = {"Dealer Name":dealer_name,\
             "Year-make-model":year_make_model,\
                 'KMs':km,\
                     'Tansmission':trans,\
                         'Price':price,\
                             'Comments':comment,\
                                    'Source URL': source,\
                                        'Year': year,\
                                            'Make': make,\
                                                'Model': model,\
                                                    'State Abrev': state,\
                                                        'Suburb': suburb,\
                                                            'Listing Type':listing_type,\
                                                                'Date scraped':date,\
                                                                    'Page Number':x}

        print(dealer_name)
        print(year_make_model)
        print(km)
        print(trans)
        print(price)
        print(comment)
        print(listing_type)
        print(suburb)


        # Update MongoDB
        col_cars.insert_one(car)
        #Append list of dictionaries
        cars.append(car)
    
    try:
        nextpage = browser.find_by_css('.listing-pagination-next').first
        nextpage.click()
        print('-------------------')
        print(f'page: {x} of {pages}')
        print('-------------------')
                 
    except:
        print("Scraping Complete")

# Quit browser
browser.quit()

Parkside Motor Co
2006 Mercedes-Benz C200 Kompressor 
189474 
Automatic
8990
Just arrived stunning C200 Kompressor Sport in immaculate condition this vehicle is a credit to its ...
USED Dealer ad
Maylands
Great Deal Auto
2009 Ford Ranger XL
195565 
Automatic
15500
We offer onsite finance through Australias leading automotive finance brokers. Trade-ins are ...
USED Dealer ad
Malaga
Fabcar
2004 Toyota Estima Aeras
84973 
Automatic
29888
** VIEWINGS BY APPOINTMENT ONLY **** MOBILITY WHEELCHAIR/ DISABILITY VEHICLE**2004 TOYOTA 7 SEATER ...
USED Dealer ad
Perth
Osborne Park Nissan used
2018 Dodge N/A Laramie Crew Cab SWB
106000 
Automatic
149999
THOUSANDS SPEND ON UPGRADES!! heavy duty rear suspension, fox racing shocks, EBA bull bar with L.E....
USED Dealer ad
Osborne Park
Subaru Wangara used
2015 Volkswagen Golf R
102001 
Manual
39998
LIVE THE FULL HOT HATCH EXPERIENCE.. Tastefully modified with OVER $13000 worth of mods and ...
USED Dealer ad
Wangara
Car Hub WA
2010 Ford Falcon XR6
24562

In [35]:
# Create statistics by suburb
df = pd.DataFrame(cars)
pd.to_numeric(df['Price'])
df2 = (df.groupby(['Suburb', 'Price'],as_index=False).mean().groupby('Suburb')['Price'].mean())
# mean_df = df2.Price.mean()
# mean_df = mean_df.reset_index()
# suburb_counts = df['Suburb'].value_counts().to_frame('Count').reset_index()
# suburb_avg_price = df['Price'].apply(pd.DataFrame.describe)
df2.head()
# df.head()

DataError: No numeric types to aggregate